# Customizing Plots

* Intro paragraph. Separation of presentation/content. Option trees.
* Renderers control output. utilities to control settings described shortly.

HoloViews is designed to be both highly customizable, allowing you to control how your visualizations appear, but also to enforce a strong separation between your data (with any semantically associated metadata, like type, dimension names, and description) and all options related purely to visualization.  This separation allows HoloViews objects to be generated easily by external programs, without giving them a dependency on any plotting or windowing libraries.  It also helps make it completely clear which parts of your code deal with the actual data, and which are just about displaying it nicely, which becomes very important for complex visualizations that become more complicated than your data itself.

To achieve this separation, HoloViews stores visualization options independently from your data, and applies the options only when rendering the data to a file on disk, a GUI window, or an IPython notebook cell.

This tutorial gives an overview of the different types of options available, how to find out more about them, and how to set them in both regular Python and using the IPython magic interface that is shown elsewhere in the tutorials.

Notes:



## Visual options and styling

* Intro paragraph explaining 3 types of option.
  + plot options
  + style options
  + normalization options

HoloViews provides three categories of visualization options that can be set by the user.  In this section we will first describe the different kinds of options, then later sections show you how to list the supported options of each type for a given HoloViews object or class, and how to change them in Python or IPython.

#### ``style`` options:

``style`` options are passed directly to the underlying rendering backend that actually draws the plots, allowing you to control the details of how it behaves.  The default backend is matplotlib, but there are other backends either using matplotlib's options (e.g. ``mpld3``), or their own sets of options (e.g. [``bokeh``](Bokeh_Backend) ).

For whichever backend has been selected, HoloViews can tell you which options are supported, but you will need to see the plotting library's own documentation (e.g. [matplotlib](http://matplotlib.org/contents.html), [bokeh](http://bokeh.pydata.org)) for the details of their use.

HoloViews has been designed to be easily extensible to additional backends in the future, such as [Plotly](https://github.com/ioam/holoviews/pull/398), Cairo, VTK, or D3.js, and if one of those backends were selected then the supported style options would differ.

#### ``plot`` options:

Each of the various HoloViews plotting classes declares various [Parameters](http://ioam.github.io/param) that control how HoloViews builds the visualization for that type of object, such as plot sizes and labels.  HoloViews uses these options internally; they are not simply passed to the underlying backend.  HoloViews documents these options fully in its online help and in the [Reference Manual](http://holoviews.org/Reference_Manual).  These options may vary for different backends in some cases, depending on the support available both in that library and in the HoloViews interface to it, but we try to keep any options that are meaningful for a variety of backends the same for all of them.

#### ``norm`` options:

``norm`` options are a special type of plot option that are applied orthogonally to the above two types, to control normalization.  Normalization refers to adjusting the properties of one plot relative to those of another.  For instance, two images normalized together would appear with relative brightness levels, with the brightest image using the full range black to white, while the other image is scaled proportionally.  Two images normalized independently would both cover the full range from black to white.  Similarly, two axis ranges normalized together will expand to fit the largest range of either axis, while those normalized separately would cover different ranges.

There are currently only two ``norm`` options supported, ``axiswise`` and ``framewise``, but they can be applied to any of the various object types in HoloViews to specify a huge range of different normalization options.

For a given category or group of HoloViews objects, if ``axiswise`` is True, normalization will be computed independently for all items in that category that have their own axes, such as different ``Image`` plots or ``Curve`` plots. If ``axiswise`` is False, all such objects are normalized together.

For a given category or group of HoloViews objects, if ``framewise`` is True, normalization of any ``HoloMap`` objects included is done independently per frame rendered -- each frame will appear as it would if it were extracted from the ``HoloMap`` and plotted separately.  If ``framewise`` is False (the default), all frames in a given ``HoloMap`` are normalized together, so that you can see strength differences over the course of the animation.

As described below, these options can be controlled precisely and in any combination to make sure that HoloViews displays the data of most interest, ignoring irrelevant differences and highlighting important ones.


### Option specification syntax

* Intro to idea of two formats and group/label specificity.


#### Dictionary format

#### String format

### %opts

* %opts as a way of specifying things globally.
* Simple example

The ``%opts`` "line" magic (with one ``%``) works just the same as the ``%%opts`` "cell" magic, but it changes the global default options for all future cells, allowing you to choose a new default colormap, line width, etc.

Apart from its brevity, a big benefit of using the IPython magic syntax ``%%opts`` or ``%opts`` is that it is fully tab-completable.  Each of the options that is currently available will be listed if you press ``<TAB>`` when you are ready to write it, which makes it much easier to find the right parameter.  Of course, you will still need to consult the full ``holoviews.help`` documentation (described above) to see the type, allowable values, and documentation for each option, but the tab completion should at least get you started and is great for helping you remember the list of options and see which options are available.

You can even use the succinct IPython-style specification directly in your Python code if you wish, but it requires the external [pyparsing](https://pypi.python.org/pypi/pyparsing) library (which is already available if you are using matplotlib):

### %%opts

* Simple example with group/label. Show that 1. styling local to that cell display 2. changes persist.

The ``%%opts`` magic works like the pure-Python option for associating options with an object, except that it works on the item in the IPython cell, and it affects the item directly rather than making a copy or applying only in scope.  Specifically, it assigns a new ID number to the object returned from this cell, and makes a new ``OptionTree`` containing the options for that ID number.

If the same ``layout`` object is used later in the notebook, even within a complicated container object, it will retain the options set on it.

The options accepted are just the same as for the Python version, but specified more succinctly:

``%%opts`` *target-specification*  ``style(``*styleoption*``=``*val* ...``) plot[``*plotoption*``=``*val* ...``] norm{+``*normoption* ``-``*normoption*...``}``

Here *key* lets you specify the object type (e.g. ``Image``), and optionally its ``group`` (e.g. ``Image.Function``) or even both ``group`` and ``label`` (e.g. ``Image.Function.Sine``), if you want to control options very precisely. There is also an even further abbreviated syntax, because the special bracket types alone are enough to indicate which category of option is specified:

``%%opts`` *target-specification*  ``(``*styleoption*``=``*val* ...``) [``*plotoption*``=``*val* ...``] {+``*normoption* ``-``*normoption* ...``}``

Here parentheses indicate style options, square brackets indicate plot options, and curly brackets indicate norm options (with ``+axiswise`` and ``+framewise`` indicating True for those values, and ``-axiswise`` and ``-framewise`` indicating False).  Additional *target-specification*s and associated options of each type for that *target-specification* can be supplied at the end of this line.  This ultra-concise syntax is used throughout the other tutorials, because it helps minimize the code needed to specify the plotting options, and helps make it very clear that these options are handled separately from the actual data.

Here we demonstrate the concise syntax by customizing the style and plot options of the ``Curve`` in the layout:

### hv.opts

* string format or dict format.



### Option trees

* All these mechanisms affect option trees which can be used directly typically at the library level.
* Explain inheritance


## Customizing output

Plot sizes, backends, output formats and saving to file. There are multiple ways to control these things.


### hv.extension


### %output



### %%output



### hv.output
